In [3]:
import json
import numpy as np
import requests
from geopy.distance import geodesic
from itertools import product
import utm
import time

# url = 'https://sk4a447dkf.execute-api.us-east-1.amazonaws.com/default/localize'
# json_data = requests.get(url).json()
# data = json_data['objects']

# cluster_objects = [(cluster['cluster_latitude'], cluster['cluster_longitude']) for cluster in data ]
with open('houston.json') as cluster_data:
    json_data = json.load(cluster_data)
cluster_objects = []
for cluster in json_data:
    cluster_objects.append(utm.to_latlon(cluster['A'], cluster['B'], 15, "R"))
    
with open('Pole.geojson') as data:
    pole_data = json.loads(data.read())

poles = [(pole['geometry']['coordinates'][1],pole['geometry']['coordinates'][0]) for pole in pole_data['features']]
start = time.time()
for cluster in cluster_objects:
    distances = []
    for pole in poles:
        distances.append(geodesic(pole, cluster).km)
    print("cluster location -->", cluster, " and the nearest pole number -->", distances.index(min(distances)))

end = time.time()
print("Total time taken to run the algorithm is ",(end-start)*1000)

cluster location --> (29.760192983939383, -95.35561792706343)  and the nearest pole number --> 63
cluster location --> (29.758972895810967, -95.35557190210065)  and the nearest pole number --> 63
cluster location --> (29.746989911147924, -95.37784098771208)  and the nearest pole number --> 257
cluster location --> (29.765533935106074, -95.37700499059427)  and the nearest pole number --> 147
cluster location --> (29.759415945319642, -95.35397486246876)  and the nearest pole number --> 63
cluster location --> (29.74934497364246, -95.37767794933187)  and the nearest pole number --> 252
cluster location --> (29.76461989315768, -95.37515496841688)  and the nearest pole number --> 144
cluster location --> (29.76438290138207, -95.37709298514051)  and the nearest pole number --> 141
cluster location --> (29.750490941221106, -95.35595187693775)  and the nearest pole number --> 94
cluster location --> (29.751810951186748, -95.3553819034064)  and the nearest pole number --> 94
Total time taken to

In [28]:
from sklearn.metrics.pairwise import haversine_distances
pole_array = []
cluster_array = []
for pole in poles:
    pole_array.append(np.array(pole))
pole_array = np.array(pole_array)

for cluster in cluster_objects:
    cluster_array.append(np.array(cluster))
cluster_array = np.array(cluster_array)
start = time.time()
result = haversine_distances(np.radians(cluster_array), np.radians(pole_array))
result.argmin(axis=1)
end = time.time()
print("Running time of the algorithm", (end-start)*1000)
result.argmin(axis=1)

Running time of the algorithm 1.6279220581054688


array([ 63,  63, 257, 147,  63, 252, 144, 141,  94,  94])

In [30]:
# Image clustering using 
url = 'https://sk4a447dkf.execute-api.us-east-1.amazonaws.com/default/localize'
json_data = requests.get(url).json()
data = json_data['objects']
images = []
for cluster in data:
    images.extend([cluster_image['image'] for cluster_image in cluster['cluster_objects']])


In [ ]:
import wget
for image in images:
    wget.download(image)

In [29]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
from sklearn.cluster import KMeans
import os, shutil, glob, os.path
from PIL import Image as pil_image
image.LOAD_TRUNCATED_IMAGES = True 
model = VGG16(weights='imagenet', include_top=False)

# Variables
imdir = '/home/aswin/Documents/SmartCity-Localization/images'
targetdir = "/home/aswin/Documents/SmartCity-Localization/images/output/"
number_clusters = 6
# Loop over files and get features
filelist = glob.glob(os.path.join(imdir, '*.jpeg'))
filelist.sort()
featurelist = []
for i, imagepath in enumerate(filelist):
    print("    Status: %s / %s" %(i, len(filelist)), end="\r")
    img = image.load_img(imagepath, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = np.array(model.predict(img_data))
    featurelist.append(features.flatten())

# Clustering
kmeans = KMeans(n_clusters=number_clusters, random_state=0).fit(np.array(featurelist))

# Copy images renamed by cluster 
# Check if target dir exists
try:
    os.makedirs(targetdir)
except OSError:
    pass
# Copy with cluster name
print("\n")
for i, m in enumerate(kmeans.labels_):
    print("    Copy: %s / %s" %(i, len(kmeans.labels_)), end="\r")
    shutil.copy(filelist[i], targetdir + str(m) + "_" + str(i) + ".jpg")

    Status: 50 / 51



In [ ]:
# Network analysis